# Bike Sharing Demand

Bu proje, bisiklet paylaşım sistemlerinin talebini tahmin etmeye yönelik bir makine öğrenimi yarışmasıdır. Amacımız, belirli bir zaman diliminde bisiklet kiralama talebini etkileyen faktörleri analiz ederek doğru tahminler yapmaktır. Bu, şehir içi ulaşımın daha verimli hale getirilmesine ve bisiklet paylaşım sistemlerinin optimize edilmesine olanak tanır.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_log_error
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
for df in [train, test]:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['hour'] = df['datetime'].dt.hour
    df['day'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month
    df['year'] = df['datetime'].dt.year
    df['weekday'] = df['datetime'].dt.weekday

In [6]:
train['count'] = np.log1p(train['count'])  # log(y + 1)

In [7]:
train = train[train['count'] < np.log1p(1000)]  # 1000+ değerleri kaldır

In [8]:
train['temp_humidity'] = train['temp'] * train['humidity']
test['temp_humidity'] = test['temp'] * test['humidity']

In [9]:
train['season_hour'] = train['season'].astype(str) + "_" + train['hour'].astype(str)
test['season_hour'] = test['season'].astype(str) + "_" + test['hour'].astype(str)

In [10]:
categorical = ['season', 'holiday', 'workingday', 'weather', 'hour', 'month', 'year', 'weekday', 'season_hour']

In [11]:
# Veriyi hazırlama
X = train.drop(['datetime', 'casual', 'registered', 'count'], axis=1)
y = train['count']
X = pd.get_dummies(X, columns=categorical)

In [12]:
# LightGBM model
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 40,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

lgb_train = lgb.Dataset(X, y)
model = lgb.train(params, lgb_train, num_boost_round=1000)

In [13]:
# GridSearch ile optimizasyon (Raj'ın yaklaşımı)
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 500, 1000]
}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror')
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)
best_model = grid_search.best_estimator_

In [14]:
# Test verisi için özellikler
test_processed = pd.get_dummies(test.drop('datetime', axis=1), columns=categorical)
test_processed = test_processed.reindex(columns=X.columns, fill_value=0)

In [15]:
preds = model.predict(test_processed)
final_preds = np.expm1(preds)  # Ters log dönüşümü

# Dosyayı kaydet
submission = pd.DataFrame({'datetime': test['datetime'], 'count': final_preds})
submission.to_csv('improved_submission.csv', index=False)

# Sonuç Paragrafı

Bisiklet paylaşım talebini tahmin etmek için geliştirdiğimiz modelin veriyi işleme ve modelleme aşamalarını kapsamaktadır. İlk olarak, veri setindeki tarih bilgileri ayrıştırılarak saat, gün, ay, yıl ve haftanın günü gibi yeni özellikler oluşturulmuştur. Ardından, bisiklet kiralama sayısının logaritması alınarak veri dağılımı normalize edilmiştir.

Modelin eğitimi için LightGBM kullanılmış ve hiperparametre optimizasyonu için GridSearchCV ile XGBoost modeli üzerinde çalışmalar yapılmıştır. Sonuç olarak, elde edilen tahminler ile Kaggle'daki yarışma skorumuz **0.38706** olarak belirlenmiştir. Bu sonuç, fena değil; bisiklet paylaşım sistemlerinin talep tahminlerinin daha doğru ve etkili bir şekilde yapılabileceğini göstermektedir. Elde edilen sonuçlar, veri bilimi ve makine öğrenimi konularında önemli bir ilerleme kaydettiğimizi gösteriyor.